# DSCI: 525 Milestone 1 - Group 8

### Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

C:\Users\SantiagoRugelesSch\miniconda3\envs\525\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


### Downloading the data

In [3]:
%pwd

'C:\\Users\\SantiagoRugelesSch\\DSCI_525\\525-group8\\notebooks'

In [4]:
%cd ~/DSCI_525/525-group8

C:\Users\SantiagoRugelesSch\DSCI_525\525-group8


In [13]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "dailyrainfall/"

In [14]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [15]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

Wall time: 2min 36s


In [16]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

Wall time: 1min 27s


### Combining data CSVs

In [5]:
#Test to check the command 
use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max","rain (mm/day)"]
BCC_df = pd.read_csv("dailyrainfall/BCC-CSM2-MR_daily_rainfall_NSW.csv", usecols=use_cols, parse_dates=["time"])
BCC_df

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-35.887649,-34.766162,141.1875,142.3125,4.265520e-17
1,1889-01-02 12:00:00,-35.887649,-34.766162,141.1875,142.3125,2.728501e-20
2,1889-01-03 12:00:00,-35.887649,-34.766162,141.1875,142.3125,5.221255e-24
3,1889-01-04 12:00:00,-35.887649,-34.766162,141.1875,142.3125,2.869350e-17
4,1889-01-05 12:00:00,-35.887649,-34.766162,141.1875,142.3125,2.290091e-15
...,...,...,...,...,...,...
3035335,2014-12-27 12:00:00,-30.280211,-29.158723,152.4375,153.5625,1.218327e+01
3035336,2014-12-28 12:00:00,-30.280211,-29.158723,152.4375,153.5625,6.209972e-01
3035337,2014-12-29 12:00:00,-30.280211,-29.158723,152.4375,153.5625,3.562820e-04
3035338,2014-12-30 12:00:00,-30.280211,-29.158723,152.4375,153.5625,4.175968e+01


In [13]:
%%time
%memit
# Join all data together
#
import pandas as pd
use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max","rain (mm/day)"]
files = glob.glob('dailyrainfall/*.csv')
df = pd.DataFrame(columns = use_cols)
for file in files:
    if len(re.findall(r'NSW', file)) > 0: # Used to ignore rainfal SYD as it misses most columns
        print(file)
        df2 = (pd.read_csv(file,usecols=use_cols, parse_dates=["time"]).
               assign(model=re.findall(r'\\(.+?)_', file)[0]))
        df = pd.concat([df, df2], axis=0)

#df.to_csv("figshareairline/combined_data.csv")

peak memory: 1245.37 MiB, increment: 0.61 MiB
dailyrainfall\ACCESS-CM2_daily_rainfall_NSW.csv
dailyrainfall\ACCESS-ESM1-5_daily_rainfall_NSW.csv
dailyrainfall\AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
dailyrainfall\BCC-CSM2-MR_daily_rainfall_NSW.csv
dailyrainfall\BCC-ESM1_daily_rainfall_NSW.csv
dailyrainfall\CanESM5_daily_rainfall_NSW.csv
dailyrainfall\CMCC-CM2-HR4_daily_rainfall_NSW.csv
dailyrainfall\CMCC-CM2-SR5_daily_rainfall_NSW.csv
dailyrainfall\CMCC-ESM2_daily_rainfall_NSW.csv
dailyrainfall\EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
dailyrainfall\FGOALS-f3-L_daily_rainfall_NSW.csv
dailyrainfall\FGOALS-g3_daily_rainfall_NSW.csv
dailyrainfall\GFDL-CM4_daily_rainfall_NSW.csv
dailyrainfall\GFDL-ESM4_daily_rainfall_NSW.csv
dailyrainfall\INM-CM4-8_daily_rainfall_NSW.csv
dailyrainfall\INM-CM5-0_daily_rainfall_NSW.csv
dailyrainfall\KIOST-ESM_daily_rainfall_NSW.csv
dailyrainfall\MIROC6_daily_rainfall_NSW.csv
dailyrainfall\MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
dailyrainfall\MPI-ESM1-2-HR_daily_r

### Load the combined CSV to memory and perform a simple EDA

In [12]:
df.shape

(55385460, 7)

### Perform a simple EDA in R

In [6]:
# Rachel and Rui